In [1]:
import sys
sys.path.append('/home/ec2-user/CpGPython/code/')
import deep_network_estimator as dne
import Logger
import pandas as pd
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
import numpy as np

In [1]:
sys.path

['',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python36.zip',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/lib-dynload',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages',
 '/home/ec2-user/anaconda3/envs/py3.6/lib/python3.6/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython',
 '/home/ec2-user/anaconda3/envs/py35tf/lib/python3.5/site-packages/']

In [5]:
from importlib import reload
reload(dne)

<module 'deep_network_estimator' from '/home/ec2-user/CpGPython/code/deep_network_estimator.py'>

In [2]:
home='/home/ec2-user/CpGPython/'
log_dir = home+'logs/'
logger = Logger.Logger(log_dir,False).get_logger()
with pd.HDFStore(home+'data/selected_features','r') as h5s:
    train_x =h5s['train_x'] 
    train_label = h5s['train_label'] 
    test_x = h5s['test_x'] 
    test_label = h5s['test_label']   
    sample_weights_train = h5s['sample_weights_train'] 
    sample_weights_test = h5s['sample_weights_test'] 
train_x['sample_weights'] = sample_weights_train
test_x['sample_weights'] = sample_weights_test

In [3]:
train_num = len(train_x)
params = {'batch_normalization':True,'l2_reg':0.01,'n_classes':len(train_label.unique()),'hidden_layers':[int(train_num*1.5),int(train_num*1),int(train_num*0.5)],'drop_out':0.5,'weight_factor':1.5,'steps':2000,'batch_size':30,'scoring':'precision'}
estimator = dne.tensor_DNN(home=home,**params)

In [5]:
from sklearn.model_selection import cross_val_score

In [6]:
results = cross_val_score(estimator,train_x,train_label,cv=5,scoring='precision')

Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)


TypeError: Singleton array 158 cannot be considered a valid collection.

In [4]:
estimator.fit(train_x,train_label)

Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)


In [4]:

param_grid = {'batch_normalization':[True],'l2_reg':np.linspace(0.001,0.03,10),'n_classes':[len(train_label.unique())],'hidden_layers':[[int(train_num*1.5),int(train_num*1),int(train_num*0.5)]],'drop_out':np.linspace(0.1,0.5,10),'weight_factor':np.linspace(1,2,10),'steps':np.linspace(200,2000,10,dtype='i8'),'batch_size':[30],'scoring':['precision']}
search = RandomizedSearchCV(estimator,param_distributions=param_grid,n_iter=50,cv=5)

In [5]:
search.fit(train_x,train_label)

Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)
Tensor("IteratorGetNext:176", s

RandomizedSearchCV(cv=5, error_score='raise',
          estimator=tensor_DNN(batch_normalization=True, batch_size=30, drop_out=0.5,
      hidden_layers=[3169, 2113, 1056], l2_reg=0.01, n_classes=2,
      scoring='precision', steps=2000, weight_factor=1.5),
          fit_params=None, iid=True, n_iter=50, n_jobs=1,
          param_distributions={'batch_normalization': [True], 'l2_reg': array([0.001  , 0.00422, 0.00744, 0.01067, 0.01389, 0.01711, 0.02033,
       0.02356, 0.02678, 0.03   ]), 'n_classes': [2], 'hidden_layers': [[3169, 2113, 1056]], 'drop_out': array([0.1    , 0.14444, 0.18889, 0.23333, 0.27778, 0.32222, 0....400,  600,  800, 1000, 1200, 1400, 1600, 1800, 2000]), 'batch_size': [30], 'scoring': ['precision']},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [12]:
predicts = estimator.predict_prob(test_x)

In [5]:
eval_results = estimator.evaluate(test_x,test_label)

Tensor("IteratorGetNext:176", shape=(?,), dtype=float64)


In [6]:
eval_results

{'auc': 0.5, 'precision': 0.0, 'recall': 0.0}